# FIT5148 - Distributed Databases and Big Data

# Assignment 2 - Task 2 Solution Workbook


**Student Details**
- Name: Pushan Mukerjee
- Student ID: 29052971


In [73]:
#Libraries 

import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd #for data load 
import json #for data load
#from json import loads
#from collections import OrderedDict

#import csv #for data load
#import datetime as dt #for datetime manipulation
#import csv #for data load
#import multiprocessing as mp # For parallel processing
#import matplotlib.pyplot as plt #For profiling the Fire data. 
#import numpy as np #For profiling the Fire Data (getting the 25th and 75th percentile)
#import os #for getting the process id in parallel processing
#import time #to record the start time and end time of process execution
#import sys #to use sys.maxsize in k-way merge


## Q1) Data Model

There is a 1:Many relationship between climate and fire. One climatic day contains 0 or many fires, whereas one fire belongs to only one climatic day, with Date being the join column. 

The model chosen was an **Embedding** model that embedded the one related climate object into each fire. Sample of the model is below.

**Sample:**

```
{
    "_id" : ObjectId("5baa31d4fa3c2e011bb72abe"),
	"Latitude" : -37.651,
	"Longitude" : 149.345,
	"Surface_Temperature_Kelvin" : 337.8,
	"Datetime" : "2017-12-16T00:20:53",
	"Power" : 42.2,
	"Confidence" : 82,
	"Date" : "2017-12-16",
	"Surface_Temperature_Celcius" : 64,
	"Climate" : 
		{   "Station" : 948702,
			"Date" : "2017-12-16",
			"Air_Temperature_Celcius" : 18,
			"Relative_Humidity" : 53.7,
			"WindSpeed_knots" : 9,
			"Max_Wind_Speed" : 13,
			"Precipitation" : "0.00I"
		}
}
```
**Note:** The order of the JSON elements is unspecified and is put into alphabetical order by **loads** package.
Hence in Mongodb, the above document may appear as per the below sample: 

```
{'Climate': {'Air_Temperature_Celcius': 28,
  'Date': '2017-12-27',
  'Max_Wind_Speed': 15.9,
  'Precipitation ': ' 0.00I',
  'Relative_Humidity': 58.3,
  'Station': 948702,
  'WindSpeed_knots': 9.3},
 'Confidence': 78,
 'Date': '2017-12-27',
 'Datetime': '2017-12-27T04:16:51',
 'Latitude': -37.966,
 'Longitude': 145.051,
 'Power': 26.7,
 'Surface_Temperature_Celcius': 68,
 'Surface_Temperature_Kelvin': 341.8,
 '_id': ObjectId('5baf7d2893436908f141b36e')}
```


**Justification:**

The model chosen was an Embedding model that embedded the climate information into each fire. Due to the 1:Many relationship between Climate and Fire, the model can be limited to only one embedded Climate object per fire, hence eliminating the chance of the 16MB document limit from breaching. As a result, this model enables all information about a single fire to be retrieved in a single seek, which is faster than referencing.   In addition, the model is most simple and fit for purpose given the queries in this task where some queries do not even need adjoined climate data. Yet all queries need fire data.    

Another option would have been to use a Referencing Model and reference the fire information within the Climate collection. The issue in this case, is you cannot identify a single fire by the joining column Date. So this would require storing an array of fire datestamps within a climate object in order to reference each fire. Additional overhead for not much benefit since in this task, a lot of the queries don't need joins and hence faster to retrieve all information contained within a single document.

An embedded option, where all fire instances are embedded within a single climate object is worse than the above two, as there are many fires on a particular climatic day, so the chances of breaching the 16MB limit for a single document would be high. 

## Q2) Loading Documents into the Data Model

Create the Client Connection to MongoDb and the new collection as2TaskB. Then load the datasets into the collection using the new model.

In [74]:
client = MongoClient () #defining the Mongodb client.
db = client.fit5148_db #defining the db 
collection = db.as2TaskB #define a new collection as2TaskB. This will store the new data model described above.

climateData = [] #climate data array to store in memory
fireData = [] #fire data array to store in memory

result = collection.drop() #Ensure that the collection does not already exist in Mongo db

fireDf = pd.read_csv("FireData-Part1.csv") #Read the FireData-Part1.csv into a dataframe
climateDf = pd.read_csv("ClimateData-Part1.csv") #Read the ClimateData-Part1.csv into a dataframe

mergedDf = pd.merge(fireDf, climateDf, on='Date', how='inner') #Join fire and climate dataframes by Date

#Create a dataframe that stores the climate columns for each merged fire-climate record. 
#This dataframe will be used to create an embedded climate JSON Object
embeddedDf = mergedDf[['Station', 'Date', 'Air_Temperature_Celcius', 'Relative_Humidity', 'WindSpeed_knots', 'Max_Wind_Speed', 'Precipitation ']]

#For each row of fireData dataframe, create a new column called 'Climate'
#Create an embedded Climate JSON objects from the Embedded Dataframe, 
#Store the embedded Climate JSON Objects in the Climate column of fireData dataframe
fireDf["Climate"] = json.loads(embeddedDf.to_json(orient='records'), object_pairs_hook=OrderedDict)

#Convert the fireData dataframe into a set of JSON objects, with each object containing an embedded the climate JSON object.
fireRecords = json.loads(fireDf.to_json(orient='records'), object_pairs_hook=OrderedDict)

#Insert the fireData JSON objects (with the embedded Climate object) into the Collection as2TaskB
result = collection.insert_many(fireRecords)


Confirm the number of JSON documents inserted into the Collection (should be 2668 as per the number of fire records)

In [75]:
numRecs = collection.count() #count the number of JSON documents inserted into the Collection
numRecs #print the count. Should be 2668.


2668

Printing out a sample JSON document in the collection (first document) 

In [77]:
collection.find()[0]

{'Climate': {'Air_Temperature_Celcius': 28,
  'Date': '2017-12-27',
  'Max_Wind_Speed': 15.9,
  'Precipitation ': ' 0.00I',
  'Relative_Humidity': 58.3,
  'Station': 948702,
  'WindSpeed_knots': 9.3},
 'Confidence': 78,
 'Date': '2017-12-27',
 'Datetime': '2017-12-27T04:16:51',
 'Latitude': -37.966,
 'Longitude': 145.051,
 'Power': 26.7,
 'Surface_Temperature_Celcius': 68,
 'Surface_Temperature_Kelvin': 341.8,
 '_id': ObjectId('5baf885893436908f141d2b7')}

In [69]:
#climateRecords = json.loads(embeddedDf.to_json(orient='records'))
#records_climate[0]

#for i in range(numFires):
#        fireObj = fireRecords[i]
#        climateObj = climateRecords[i]
#        doc = dict(fireObj + {'climate:' + climateObj})
#        pprint(doc)

        
        
        #       fireObj = db.fire.find({})
#       fireObj.climate = []
        #date = fireObj["Date"]
#        climateObj = db.climate.find({"Date":date})[0]    
#        pprint(type(fireObj))
        #fireObj[i].climate = climateDoc
        #climateCursor.forEach(function(climate){ #for each climate rec in the climateCursor
        #    fireObj.climate.push(climate) #push that climate record into the fireObj.climate array
        #})
    #pprint()

#pprint(fireObj[1])    




#pattern = r"({|,)(?:\s*)(?:')?([A-Za-z_$\.][A-Za-z0-9_ \-\.$]*)(?:')?(?:\s*):"

#records = map(lambda x, y: dict(x.items() + {'climate:': json.loads(y.to_json(orient='records'))}.items(), [fireRecords, climateRecords]))              
#records

#records = map(lambda x:
#  dict(x.items() +
#    {
#      'criteria': json.loads(
#        re.sub(pattern, "\\1\"\\2\":", x['criteria'])
#      )
#    }.items()
#  ),
#  records
#)

#records




#with open('ClimateData.csv') as climate_file:
#    climate_reader = csv.reader(climate_file, delimiter=',') #read in the ClimateData.csv
#    line_count = 0
#    for row in climate_reader:
#        if line_count == 0: #Header row
#            climateHeaders = row #create header row
#            line_count += 1 #go to next row in file
#        else: #data row
#            climateData.append(row) #insert row into array (memory) 
#            line_count += 1 #go to next row in file




#fireObj = db.fire.find({}) #find the fires
#fireObj.climate = [] #create an empty array to store associated climate data



#df = pd.read_csv("file.csv")
#records = json.loads(df.to_json(orient='records'))

#pattern = r"({|,)(?:\s*)(?:')?([A-Za-z_$\.][A-Za-z0-9_ \-\.$]*)(?:')?(?:\s*):"

#records = map(lambda x:
#  dict(x.items() +
#    {
#      'criteria': json.loads(
#        re.sub(pattern, "\\1\"\\2\":", x['criteria'])
#      )
#    }.items()
#  ),
#  records
#)





#for doc in fireObj:
#    date = fireObj[1]
    #print(date) 
#    pprint(date)

#var studentList = db.users.find({"completed_units":unitCode})    
#date = "2017-12-15"
#cursor = db.climate.find({"Date":date})

#db.test.find()[50]

#for doc in cursor:
#    pprint(doc)

#climate = doc.find.({"Date":unitCode})
#var climate = db.climate.find({"Date":unitCode}) // find students having the unitCode
    
#collection = db.fit5148_collection
#result = collection.insert_many(newTours)

#cursor = montours.find({})

#cursor = fireObj
#for document in cursor: 
#    pprint(document)
#    collection.insert_many(newTours)